In [1]:
import numpy as np
import pandas as pd
import statsmodels as sms
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection

import seaborn as sns # for data visualization
sns.set_style("whitegrid")

from dateutil.relativedelta import *
from pandas.tseries.offsets import *

pd.set_option('display.max_columns', None)

In [3]:
gkx = pd.read_csv('~/misp_data/gkx_mlfactors.csv')

In [7]:
gkx.tail()

,permno,DATE,mvel1,beta,betasq,chmom,dolvol,idiovol,indmom,mom1m,mom6m,mom12m,mom36m,pricedelay,turn,absacc,acc,age,agr,bm,bm_ia,cashdebt,cashpr,cfp,cfp_ia,chatoia,chcsho,chempia,chinv,chpmia,convind,currat,depr,divi,divo,dy,egr,ep,gma,grcapx,grltnoa,herf,hire,invest,lev,lgr,mve_ia,operprof,orgcap,pchcapx_ia,pchcurrat,pchdepr,pchgm_pchsale,pchquick,pchsale_pchinvt,pchsale_pchrect,pchsale_pchxsga,pchsaleinv,pctacc,ps,quick,rd,rd_mve,rd_sale,realestate,roic,salecash,saleinv,salerec,secured,securedind,sgr,sin,sp,tang,tb,aeavol,cash,chtx,cinvest,ear,nincr,roaq,roavol,roeq,rsup,stdacc,stdcf,ms,baspread,ill,maxret,retvol,std_dolvol,std_turn,zerotrade,sic2,date
3760203,93428,20161230,1.250976e+06,1.595315,2.545028,-0.137348,14.916210,0.043201,0.016401,-0.001203,-0.047018,0.037972,0.506586,-0.000314,2.258305,0.085975,-0.085975,6.0,-0.330296,0.252853,-0.166734,0.065594,2.513108,0.043555,-0.002498,0.024946,0.004733,0.092304,0.000000,0.115472,1.0,2.735154,0.934244,0.0,0.0,0.000000,0.167011,0.000174,0.492449,1.224218,0.064913,0.031524,0.438293,0.030301,0.326122,0.490100,-5945.594495,0.095219,0.005372,-0.088348,0.064177,-0.104894,-0.026557,0.064177,NaN,-0.035934,0.067197,NaN,-29.531939,4.0,2.735154,0.0,0.059079,0.217861,0.123249,0.050598,1.122611,NaN,2.579181,0.000000,0.0,0.285838,0.0,0.271177,0.567397,4.730672,1.556535,0.396779,0.000443,-0.010900,0.000839,0.0,-0.002547,0.011983,-0.005871,0.014789,0.198895,0.261325,6.0,0.029302,1.010214e-09,0.027604,0.017847,0.490189,5.347476,8.664791e-09,73.0,2016-12-31
3760204,93429,20161230,5.600537e+06,0.298267,0.088963,0.206434,16.319190,0.026198,-0.071728,0.093973,0.000143,-0.112178,0.429553,-0.050955,2.355625,0.104737,-0.104737,6.0,-0.002310,0.049733,-0.560291,1.580322,48.338808,0.046039,-0.063371,0.250537,-0.024074,-0.001802,0.000000,0.058067,0.0,2.758463,0.708919,0.0,0.0,0.013783,0.038302,0.038483,0.953897,0.372022,NaN,0.065747,0.084615,-0.361367,0.023490,-0.064939,-2310.254433,1.381972,NaN,-0.511062,-0.167569,0.274751,0.007244,-0.167569,NaN,-0.033008,NaN,NaN,-0.196344,6.0,2.758463,0.0,NaN,NaN,0.319250,0.834527,6.205637,NaN,6.601729,NaN,0.0,0.028061,0.0,0.119106,0.535097,-0.987861,0.648429,0.272223,0.011394,-0.009761,-0.029331,0.0,0.075820,0.020948,0.189489,0.005192,0.167686,0.169486,6.0,0.016001,1.093471e-10,0.040887,0.010437,0.539155,6.252920,8.275139e-09,62.0,2016-12-31
3760205,93433,20161230,2.473625e+04,2.324893,5.405128,0.328248,9.975208,0.185537,-0.002451,0.279070,-0.358209,-0.683824,0.634616,-0.427515,2.053363,0.111051,0.111051,6.0,0.513058,-1.331854,-2.127875,-1.214824,30.645440,-0.134452,-0.132299,-0.316830,0.888516,-0.931909,0.000000,-15.764856,0.0,6.239234,0.050468,0.0,0.0,0.135075,0.271181,-0.111818,0.003974,5.658716,0.177266,0.111781,-0.572581,-0.368032,0.052763,-0.575264,-1487.531806,NaN,0.034402,45.976956,2.022019,-0.782890,-0.060716,2.022019,NaN,-0.005155,-0.322558,NaN,0.202426,4.0,6.239234,0.0,NaN,NaN,NaN,0.134936,0.038522,NaN,1.531250,1.000000,1.0,-0.842288,0.0,0.001089,0.558198,-0.923297,1.473455,0.052961,0.000000,-16.900000,0.083610,0.0,-0.182378,0.120312,-0.301570,0.001157,67.959352,110.846951,2.0,0.149511,3.889972e-07,0.405473,0.137955,1.285707,29.078756,5.014198e-09,65.0,2016-12-31
3760206,93434,20161230,8.573280e+04,0.628519,0.395036,0.020854,10.862196,0.059797,0.015187,-0.049505,0.109890,0.147727,-0.233449,-0.029555,0.550716,0.134906,-0.134906,6.0,-0.370022,0.837790,0.098907,-0.018161,-10.531395,0.169262,0.351403,0.137229,0.155842,0.795305,-0.028000,-0.291295,1.0,1.338622,0.189945,0.0,0.0,0.000000,0.007027,-0.048176,0.188891,-0.793772,0.243760,0.575073,1.000000,-0.014222,1.073284,0.864897,-5386.535078,0.058389,0.001077,2.547169,-0.201849,0.553919,0.115703,-0.095111,0.679853,0.473834,0.241107,0.758781,-4.513437,4.0,0.737878,1.0,0.028787,0.023273,0.410225,0.050914,22.972843,3.181921,3.038236,0.989722,1.0,0.575833,0.0,1.236895,0.349403,0.149519,0.389172,0.047253,-0.002037,0.031897,0.058788,0.0,0.004564,0.015642,0.009970,-0.078507,0.746650,0.860708,3.0,0.042877,8.576942e-08,0

In [6]:
gkx['date']=pd.to_datetime(gkx['DATE'], format='%Y%m%d')+MonthEnd(0)

In [8]:
ret_decomp_dcf5 = pd.read_csv('~/misp_data/decomp_dcf5y_hist_1960_2014.csv', index_col=0)
ret_decomp_dcf5['jdate_crsp'] = pd.to_datetime(ret_decomp_dcf5.jdate_crsp)

/software/python-anaconda-2020.02-el7-x86_64/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [55]:
mdf5 = gkx.merge(ret_decomp_dcf5, left_on=['date', 'permno'], right_on=['jdate_crsp', 'permno'], how='inner')

In [56]:
vals = [
 'mvel1',
 'beta',
 'idiovol',
 'indmom',
 'mom1m',
 'mom6m',
 'mom12m',
 'mom36m',
 'pricedelay',
 'age',
 'agr',
 'bm',
 'bm_ia',
 'cashpr',
 'cfp',
 'cfp_ia',
 'chatoia',
 'chcsho',
 'depr',
 'divi',
 'egr',
 'ep',
 'grcapx',
 'hire',
 'invest',
 'lev',
 'mve_ia',
 'orgcap',
 'pchcurrat',
 'pchdepr',
 'pctacc',
 'quick',
 'rd',
 'rd_mve',
 'roic',
 'secured',
 'sp',
 'tang',
 'tb',
 'cash',
 'chtx',
 'cinvest',
 'nincr',
 'roaq',
 'roavol',
 'roeq',
 'rsup',
 'ms',
 'ill',
 'std_turn',]

In [57]:
import mkl
mkl.set_num_threads(5)

def ols_coef(x,formula):
    return smf.ols(formula,data=x).fit().params

def fama_macbeth_summary(p):
    s = p.describe().T
    s['std_error'] = s['std']/np.sqrt(s['count'])
    s['tstat'] = s['mean']/s['std_error']
    return s[['mean','std_error','tstat', 'count', 'std']]

def fama_macbeth_reg_panel(regdf, xname='misp', yname='ret', 
                     csname='permno', tsname='jdate_crsp'): 
    # Panel factor setting FM reg
    gamma_cs = (regdf.groupby(tsname).apply(ols_coef,f'{yname} ~ {xname}'))
    gamma_cs = gamma_cs.rename(columns={f"{xname}":f"{xname}"})
    gamma_cs=gamma_cs[[f"{xname}"]]
    return fama_macbeth_summary(gamma_cs)

In [58]:
mdf5['ret'] = np.exp(mdf5['r_t+60'])-1
mdf5['ret_ct'] = np.exp(mdf5['r*_t+60']-1)
mdf5['ret_chg'] = np.exp(mdf5['r_chg+60'])

In [59]:
%%time
res = []
for var in vals:
    regdf = mdf5[['jdate_crsp', 'permno', 'ticker', 'ret', var]].replace([np.inf, -np.inf], np.nan).dropna()
    print(f'Ret ~ {var}, 5Yr-DCF')
    results = fama_macbeth_reg_panel(regdf, xname=var, yname='ret')
#     print(results)
    res.append(results)
#     print('')

Ret ~ mvel1, 5Yr-DCF
Ret ~ beta, 5Yr-DCF
Ret ~ idiovol, 5Yr-DCF
Ret ~ indmom, 5Yr-DCF
Ret ~ mom1m, 5Yr-DCF
Ret ~ mom6m, 5Yr-DCF
Ret ~ mom12m, 5Yr-DCF
Ret ~ mom36m, 5Yr-DCF
Ret ~ pricedelay, 5Yr-DCF
Ret ~ age, 5Yr-DCF
Ret ~ agr, 5Yr-DCF
Ret ~ bm, 5Yr-DCF
Ret ~ bm_ia, 5Yr-DCF
Ret ~ cashpr, 5Yr-DCF
Ret ~ cfp, 5Yr-DCF
Ret ~ cfp_ia, 5Yr-DCF
Ret ~ chatoia, 5Yr-DCF
Ret ~ chcsho, 5Yr-DCF
Ret ~ depr, 5Yr-DCF
Ret ~ divi, 5Yr-DCF
Ret ~ egr, 5Yr-DCF
Ret ~ ep, 5Yr-DCF
Ret ~ grcapx, 5Yr-DCF
Ret ~ hire, 5Yr-DCF
Ret ~ invest, 5Yr-DCF
Ret ~ lev, 5Yr-DCF
Ret ~ mve_ia, 5Yr-DCF
Ret ~ orgcap, 5Yr-DCF
Ret ~ pchcurrat, 5Yr-DCF
Ret ~ pchdepr, 5Yr-DCF
Ret ~ pctacc, 5Yr-DCF
Ret ~ quick, 5Yr-DCF
Ret ~ rd, 5Yr-DCF
Ret ~ rd_mve, 5Yr-DCF
Ret ~ roic, 5Yr-DCF
Ret ~ secured, 5Yr-DCF
Ret ~ sp, 5Yr-DCF
Ret ~ tang, 5Yr-DCF
Ret ~ tb, 5Yr-DCF
Ret ~ cash, 5Yr-DCF
Ret ~ chtx, 5Yr-DCF
Ret ~ cinvest, 5Yr-DCF
Ret ~ nincr, 5Yr-DCF
Ret ~ roaq, 5Yr-DCF
Ret ~ roavol, 5Yr-DCF
Ret ~ roeq, 5Yr-DCF
Ret ~ rsup, 5Yr-DCF
Ret ~ ms, 5Yr-DCF

In [60]:
res_df  = pd.concat(res)

In [61]:
res_df.head()

,mean,std_error,tstat,count,std
mvel1,-1.325632e-07,1.576636e-08,-8.407978,648.0,4.013461e-07
beta,5.941714e-02,2.901647e-02,2.047704,648.0,7.386388e-01
idiovol,2.475506e+00,1.077601e+00,2.297239,648.0,2.743123e+01
indmom,-1.415800e-01,9.854536e-02,-1.436698,648.0,2.508555e+00
mom1m,-3.000628e-01,1.218380e-01,-2.462802,648.0,3.101488e+00


In [64]:
%%time
res_ct = []
for var in vals:
    regdf = mdf5[['jdate_crsp', 'permno', 'ticker', 'ret_ct', var]].replace([np.inf, -np.inf], np.nan).dropna()
    print(f'{var}, retct')
    results = fama_macbeth_reg_panel(regdf, xname=var, yname='ret_ct')
#     print(results)
    res_ct.append(results)
#     print('')

mvel1, retct
beta, retct
idiovol, retct
indmom, retct
mom1m, retct
mom6m, retct
mom12m, retct
mom36m, retct
pricedelay, retct
age, retct
agr, retct
bm, retct
bm_ia, retct
cashpr, retct
cfp, retct
cfp_ia, retct
chatoia, retct
chcsho, retct
depr, retct
divi, retct
egr, retct
ep, retct
grcapx, retct
hire, retct
invest, retct
lev, retct
mve_ia, retct
orgcap, retct
pchcurrat, retct
pchdepr, retct
pctacc, retct
quick, retct
rd, retct
rd_mve, retct
roic, retct
secured, retct
sp, retct
tang, retct
tb, retct
cash, retct
chtx, retct
cinvest, retct
nincr, retct
roaq, retct
roavol, retct
roeq, retct
rsup, retct
ms, retct
ill, retct
std_turn, retct
CPU times: user 21min 13s, sys: 56.4 s, total: 22min 10s
Wall time: 7min 29s


In [65]:
res_ct_df  = pd.concat(res_ct)

In [66]:
res_ct_df.head()

,mean,std_error,tstat,count,std
mvel1,-1.189740e-07,1.461706e-08,-8.139397,612.0,3.616061e-07
beta,5.472220e-01,4.267876e-02,12.821882,612.0,1.055814e+00
idiovol,2.300280e+01,1.605142e+00,14.330692,612.0,3.970903e+01
indmom,-3.751400e-01,9.207910e-02,-4.074106,612.0,2.277911e+00
mom1m,-6.830200e-01,2.303922e-01,-2.964597,612.0,5.699587e+00


In [67]:
%%time
res_chg = []
for var in vals:
    regdf = mdf5[['jdate_crsp', 'permno', 'ticker', 'ret_chg', var]].replace([np.inf, -np.inf], np.nan).dropna()
    print(f'{var}, retchg')
    results = fama_macbeth_reg_panel(regdf, xname=var, yname='ret_chg')
#     print(results)
    res_chg.append(results)
#     print('')

mvel1, retchg
beta, retchg
idiovol, retchg
indmom, retchg
mom1m, retchg
mom6m, retchg
mom12m, retchg
mom36m, retchg
pricedelay, retchg
age, retchg
agr, retchg
bm, retchg
bm_ia, retchg
cashpr, retchg
cfp, retchg
cfp_ia, retchg
chatoia, retchg
chcsho, retchg
depr, retchg
divi, retchg
egr, retchg
ep, retchg
grcapx, retchg
hire, retchg
invest, retchg
lev, retchg
mve_ia, retchg
orgcap, retchg
pchcurrat, retchg
pchdepr, retchg
pctacc, retchg
quick, retchg
rd, retchg
rd_mve, retchg
roic, retchg
secured, retchg
sp, retchg
tang, retchg
tb, retchg
cash, retchg
chtx, retchg
cinvest, retchg
nincr, retchg
roaq, retchg
roavol, retchg
roeq, retchg
rsup, retchg
ms, retchg
ill, retchg
std_turn, retchg
CPU times: user 21min 14s, sys: 55.9 s, total: 22min 10s
Wall time: 7min 27s


In [68]:
res_chg_df  = pd.concat(res_chg)

In [69]:
res_chg_df.head()

,mean,std_error,tstat,count,std
mvel1,-2.779139e-07,2.627082e-08,-10.578803,612.0,6.499043e-07
beta,1.496485e+00,1.560128e-01,9.592068,612.0,3.859543e+00
idiovol,4.767409e+01,2.273777e+00,20.966913,612.0,5.625015e+01
indmom,6.616952e-01,8.097593e-01,0.817150,612.0,2.003234e+01
mom1m,6.312504e-01,7.284329e-01,0.866587,612.0,1.802043e+01


In [72]:
res_df = res_df.rename(columns={"mean":"mean_ret", "std_error":"std_error_ret",  "tstat":"tstat_ret"})
res_ct_df = res_ct_df.rename(columns={"mean":"mean_ct", "std_error":"std_error_ct",  "tstat":"tstat_ct"})
res_chg_df = res_chg_df.rename(columns={"mean":"mean_chg", "std_error":"std_error_chg",  "tstat":"tstat_chg"})

In [73]:
res_df = res_df[["mean_ret", "std_error_ret", "tstat_ret"]]
res_ct_df = res_ct_df[["mean_ct", "std_error_ct", "tstat_ct"]]
res_chg_df = res_chg_df[["mean_chg", "std_error_chg", "tstat_chg"]]

In [74]:
mm1 = res_df.merge(res_ct_df, left_index=True, right_index=True)
mm2 = mm1.merge(res_chg_df, left_index=True, right_index=True)

In [75]:
mm2.head()

,mean_ret,std_error_ret,tstat_ret,mean_ct,std_error_ct,tstat_ct,mean_chg,std_error_chg,tstat_chg
mvel1,-1.325632e-07,1.576636e-08,-8.407978,-1.189740e-07,1.461706e-08,-8.139397,-2.779139e-07,2.627082e-08,-10.578803
beta,5.941714e-02,2.901647e-02,2.047704,5.472220e-01,4.267876e-02,12.821882,1.496485e+00,1.560128e-01,9.592068
idiovol,2.475506e+00,1.077601e+00,2.297239,2.300280e+01,1.605142e+00,14.330692,4.767409e+01,2.273777e+00,20.966913
indmom,-1.415800e-01,9.854536e-02,-1.436698,-3.751400e-01,9.207910e-02,-4.074106,6.616952e-01,8.097593e-01,0.817150
mom1m,-3.000628e-01,1.218380e-01,-2.462802,-6.830200e-01,2.303922e-01,-2.964597,6.312504e-01,7.284329e-01,0.866587


In [76]:
mm2.to_csv('gkx_on_decomp_dcf5y_hist_1960_2014.csv')